In [1]:
import numpy as np
from IO import IO
from PC_denoiser import PC_denoiser
from PointFilter.Customer_Module.chamfer_distance.dist_chamfer import chamferDist
import torch
import os
import time

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


c:\Users\azhenis4\AppData\Local\anaconda3\envs\fyp_env\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: 
  warn(f"Failed to load image Python extension: {e}")


In [2]:
from tqdm import tqdm
from functools import partialmethod

tqdm.__init__ = partialmethod(tqdm.__init__, disable=True)

In [3]:
cloud_pairs = [
    ("Tetrahedron.npy", "Tetrahedron_0.005.npy"),
    ("fandisk_100K.npy", "fandisk_100K_0.005.npy"), 
    ("cube_uniform_100K.npy", "cube_uniform_100K_0.005.npy"),
    ("boxunion2_100K.npy", "boxunion2_100K_0.005.npy")
]

In [4]:
results = {}
for gt_path, noisy_path in cloud_pairs:
    results[gt_path] = {}
    gt = IO.get_arr_from(os.path.join("data", gt_path))
    noisy = IO.get_arr_from(os.path.join("data", noisy_path))
    for name in dir(PC_denoiser):
        # Get the attribute
        if name.startswith("denoise"):
            method = getattr(PC_denoiser, name)

            start_time = time.time()
            denoised = method(noisy)
            execution_time = time.time() - start_time

            loss = PC_denoiser.calculate_chamfer_distance_loss(gt, denoised)

            results[gt_path][name] = {"time": execution_time, "loss": loss}

            path = os.path.join("images", f"{gt_path[:-4]}_{name}.png")
            IO.save_visualization(cloud=denoised, save_path=path)
results

Denoise Clusters:   0%|          | 0/4 [00:00<?, ?it/s]

Denoise Clusters:   0%|          | 0/4 [00:00<?, ?it/s]

KeyboardInterrupt: 